We’ve thoroughly reviewed the Amazon dataset's background and are now shifting our focus to a closer analysis of the column review text. We will conduct some characteristics:
1. words
2. characters 
2. punctuation
3. Subject (I)
4. Uppercase letter
5. stop words

In [9]:
import pandas as pd

import string
import nltk

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/caophuong/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.corpus import stopwords

In [3]:
fake_reviews_df = pd.read_csv('/Users/caophuong/Documents/Y3S1/BT4012/Fake-Reviews-Detection/Dataset/fake reviews dataset.csv')

yelp_df = pd.read_excel('/Users/caophuong/Documents/Y3S1/BT4012/Fake-Reviews-Detection/Dataset/Yelp Labelled Review Dataset with Sentiments and Features.xlsx')

In [4]:
fake_reviews_df

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,1,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,1,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,1,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,1,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,1,Very nice set. Good quality. We have had the s...
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,0,I had read some reviews saying that this bra r...
40428,Clothing_Shoes_and_Jewelry_5,5.0,1,I wasn't sure exactly what it would be. It is ...
40429,Clothing_Shoes_and_Jewelry_5,2.0,0,"You can wear the hood by itself, wear it with ..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,1,I liked nothing about this dress. The only rea...


## FAKE REVIEWS

In [4]:
# map label

fake_reviews_df.loc[fake_reviews_df["label"] == "CG", "label"] = 1
fake_reviews_df.loc[fake_reviews_df["label"] == "OR", "label"] = 0

fake_reviews_df.rename(columns={'text_': 'text'}, inplace=True)

fake_reviews_df = fake_reviews_df.drop_duplicates('text')

In [5]:
fake_reviews_df.head()

,category,rating,label,text
0,Home_and_Kitchen_5,5.0,1,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,1,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,1,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,1,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,1,Very nice set. Good quality. We have had the s...


In [18]:
def calculate_metrics(row):
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    text = row['text']
    # Replace punctuation with spaces before splitting
    cleaned_text = ''.join([char if char not in string.punctuation else ' ' for char in text])
    words = [word.lower() for word in cleaned_text.split() if word]  # Convert to lowercase for stopword comparison
    
    # Calculate metrics
    word_count = len(words)
    characters = len(text)
    punctuation = sum(1 for char in text if char in string.punctuation)
    subject_i = sum(1 for word in text.split() if word.lower() == 'i')
    uppercase_letters = sum(1 for char in text if char.isupper())
    stopwords_count = sum(1 for word in words if word in stop_words)
    
    return pd.Series([word_count, characters, punctuation, subject_i, uppercase_letters, stopwords_count],
                    index=['words', 'characters', 'punctuation', 'subject_i', 'uppercase_letters', 'stopwords'])

In [19]:
fake_reviews_df[['words', 'characters', 'punctuation', 'subject_i', 'uppercase_letters', 'stopwords']] = fake_reviews_df.apply(calculate_metrics, axis=1)

In [21]:
fake_reviews_df['text'][0]

'Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty'

In [20]:
fake_reviews_df.head()

,category,rating,label,text,words,characters,punctuation,subject_i,uppercase_letters,stopwords
0,Home_and_Kitchen_5,5.0,1,"Love this! Well made, sturdy, and very comfor...",13,75,5,1,4,6
1,Home_and_Kitchen_5,5.0,1,"love it, a great upgrade from the original. I...",17,80,3,0,1,10
2,Home_and_Kitchen_5,5.0,1,This pillow saved my back. I love the look and...,14,67,2,1,2,7
3,Home_and_Kitchen_5,1.0,1,"Missing information on how to use it, but it i...",17,81,2,1,2,11
4,Home_and_Kitchen_5,5.0,1,Very nice set. Good quality. We have had the s...,18,85,2,0,3,11


In [22]:
fake_reviews_df.describe()

,rating,words,characters,punctuation,subject_i,uppercase_letters,stopwords
count,40412.000000,40412.000000,40412.000000,40412.000000,40412.000000,40412.000000,40412.000000
mean,4.256409,69.419455,351.365337,10.221617,2.385702,9.785039,37.589726
std,1.144538,71.854454,369.871842,13.248720,3.448892,15.825259,39.327294
min,1.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,21.000000,107.000000,3.000000,0.000000,3.000000,11.000000
50%,5.000000,40.000000,198.000000,6.000000,1.000000,5.000000,21.000000
75%,5.000000,87.000000,439.000000,12.000000,3.000000,11.000000,48.000000
max,5.000000,524.000000,2827.000000,347.000000,58.000000,805.000000,214.000000


In [23]:
label_comparison = fake_reviews_df.groupby('label')[['words', 'characters', 'punctuation', 'subject_i', 'uppercase_letters', 'stopwords']].mean()

In [24]:
print("\nComparison between correct (0) and fake (1) reviews:\n", label_comparison)


Comparison between correct (0) and fake (1) reviews:
            words  characters  punctuation  subject_i  uppercase_letters  \
label                                                                     
0      75.580213  396.973337    11.811724   1.960030          11.012614   
1      63.253206  305.716691     8.630094   2.811754           8.556370   

       stopwords  
label             
0      38.370418  
1      36.808338  
